# visualization

> Necessary scripts to visualize orbits

In [ ]:
#| default_exp visualize

In [ ]:
#| export
from nbdev.showdoc import *
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import plotly.graph_objects as go

ModuleNotFoundError: No module named 'nbdev'

In [ ]:
from orbit_generation_testing.reader import get_example_mat_data

In [ ]:
mat_data= get_example_mat_data()
mat_data.shape

# Static

In [ ]:
#| export
def visualize_static_orbits(data, time_instants, orbit_indices=None, show_legend=False):
    """
    Visualizes orbits and highlights specified time instants for every orbit in orbit_indices with data shape (num_orbits, 6, num_time_points).

    :param data: numpy.ndarray, shape (num_orbits, 6, num_time_points), containing orbit data.
    :param time_instants: list[int], time instants to highlight across selected orbits.
    :param orbit_indices: Optional[list], indices of the orbits to visualize and highlight. If None, uses all orbits.
    :param show_legend: Optional[bool], indicates whether to show the legend.
    """
    fig = plt.figure(figsize=(10, 8))
    ax = fig.add_subplot(111, projection='3d')

    if orbit_indices is None:
        orbit_indices = range(data.shape[0])

    # Validate time instants
    max_time_instants = data.shape[2]
    for time_instant in time_instants:
        if time_instant < 0 or time_instant >= max_time_instants:
            raise ValueError(f"Time instant {time_instant} is out of range.")

    # Plot all requested orbits
    for index in orbit_indices:
        if index < 0 or index >= data.shape[0]:
            print(f"Orbit index {index} is out of range.")
            continue

        X = data[index, 0, :]  # X coordinates
        Y = data[index, 1, :]  # Y coordinates
        Z = data[index, 2, :]  # Z coordinates
        ax.plot(X, Y, Z, label=f'Orbit {index}', alpha=0.5)  # Reduced alpha to emphasize highlights

        # Highlight the specified time instants for each orbit
        for time_instant in time_instants:
            # Adjust indexing for the new data shape
            posx, posy, posz = data[index, 0:3, time_instant]
            ax.scatter(posx, posy, posz, color='red', s=100, zorder=5)

    ax.set_xlabel('X')
    ax.set_ylabel('Y')
    ax.set_zlabel('Z')
    plt.title('3D Orbits Static Visualization')

    if show_legend:
        ax.legend()

    plt.show()


In [ ]:
visualize_static_orbits(data= mat_data,time_instants=[0], orbit_indices=[0,20,40])

In [ ]:
#| test "invalid time_instants raises ValueError"
from pytest import raises
time_instants = [-1, 11]  # Out of bounds
with raises(ValueError):
    visualize_static_orbits(mat_data, time_instants)

In [ ]:
#| test "invalid orbit_indices prints warning"
from unittest.mock import patch
time_instants = [0, 9]  # Valid
orbit_indices = [201]  # Out of bounds
with patch("builtins.print") as mock_print:
    visualize_static_orbits(mat_data, time_instants, orbit_indices)
    mock_print.assert_called()

# Dynamic

In [ ]:
#| export
def export_dynamic_orbits_html(data, timestamps, orbit_indices=None, filename='orbits.html', point_dict=None):
    """
    Visualize orbits in 3D and save as an interactive HTML file with a clickable legend, for data organized as (num_orbits, 6, num_time_points).
    Includes the option to add named points to the visualization and highlights the positions of each orbit at given timestamps.

    :param data: Numpy array of shape (num_orbits, 6, num_time_points)
    :param timestamps: List of int, the time instants to highlight across all orbits.
    :param orbit_indices: Optional; List of indices of the orbits to visualize
    :param filename: String, name of the file to save the HTML plot
    :param point_dict: Optional; Dictionary where keys are point names and values are the 3D coordinates of the points
    """
    num_orbits = data.shape[0]  # Adjusted for the new data shape
    if orbit_indices is None:
        orbit_indices = range(num_orbits)  # Default to all orbits

    fig = go.Figure()

    # Validate the provided timestamps
    for timestamp in timestamps:
        if timestamp < 0 or timestamp >= data.shape[2]:
            raise ValueError(f"The provided timestamp {timestamp} is out of range.")

    # Plot each orbit
    for index in orbit_indices:
        if index < 0 or index >= num_orbits:
            print(f"Orbit index {index} is out of range.")
            continue

        # Adjusted indexing for the new data shape
        X = data[index, 0, :]  # X coordinates
        Y = data[index, 1, :]  # Y coordinates
        Z = data[index, 2, :]  # Z coordinates

        fig.add_trace(go.Scatter3d(x=X, y=Y, z=Z, mode='lines',
                                   name=f'Orbit {index}',
                                   legendgroup=f'orbit{index}',
                                   showlegend=True))

        # Highlight the positions at the given timestamps
        for timestamp in timestamps:
            highlight_x = [data[index, 0, timestamp]]
            highlight_y = [data[index, 1, timestamp]]
            highlight_z = [data[index, 2, timestamp]]
            fig.add_trace(go.Scatter3d(x=highlight_x, y=highlight_y, z=highlight_z, mode='markers',
                                       marker=dict(size=5, color='red'),
                                       name=f'Highlight {index} @ {timestamp}',
                                       legendgroup=f'orbit{index}',
                                       showlegend=False))

    # Add points from the point_dict if provided
    if point_dict is not None:
        for point_name, coords in point_dict.items():
            fig.add_trace(go.Scatter3d(x=[coords[0]], y=[coords[1]], z=[coords[2]], mode='markers',
                                       marker=dict(size=5),
                                       name=point_name))

    fig.update_layout(title='3D Orbits Visualization',
                      scene=dict(xaxis_title='X',
                                 yaxis_title='Y',
                                 zaxis_title='Z'),
                      width=800, height=600,
                      legend_title="Orbits Legend",
                      clickmode='event+select')

    fig.write_html(filename)
    print(f"Visualization saved to {filename}")



In [ ]:
mu = 0.0122
points_to_add={
    'Moon': (1-mu,0,0),
    'Earth': (-mu,0,0),
    'Lagrange 1': (0.8369,0,0),
    'Lagrange 2': (1.1557,0,0)
}

In [ ]:
export_dynamic_orbits_html(data=mat_data, point_dict= points_to_add, timestamps=[0], orbit_indices=[1])

In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()